In [8]:
### lem_train2와 lem_test2를 만드는 코드
### IMPORT
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Load Raw Dataset
df_train = pd.read_csv("https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/dataset/1/drugsComTrain_raw.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/dataset/1/drugsComTest_raw.csv") 

df_train.head()

print(df_train.isnull().values.any())
print(df_test.isnull().values.any())

print(len(df_train))
print(len(df_test))

df_train = df_train.dropna(how='any').reset_index()
df_test = df_test.dropna(how='any').reset_index()

print(df_train.isnull().values.any())
print(df_test.isnull().values.any())

print(len(df_train))
print(len(df_test))
print(df_train['review'][0])

df_train['review']=df_train['review'].str.lower()
df_test['review']=df_test['review'].str.lower()
df_train['condition']=df_train['condition'].str.lower()
df_test['condition']=df_test['condition'].str.lower()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
True
True
161297
53766
False
False
160398
53471
"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"


In [9]:
# Train Set
train_list = set(df_train.index)
span_list1 = []
for i,j in enumerate(df_train['condition']):
    if '</span>' in j:
        span_list1.append(i)
# Test Set
test_list = set(df_test.index)
span_list2 = []
for i,j in enumerate(df_test['condition']):
    if '</span>' in j:
        span_list2.append(i)


In [10]:

# Train set remove
new_idx = train_list.difference(set(span_list1))
df_train = df_train.iloc[list(new_idx)].reset_index()
del df_train['index']

# Test set remove
new_idx = test_list.difference(set(span_list2))
df_test = df_test.iloc[list(new_idx)].reset_index()
del df_test['index']

In [11]:
### 알파벳과 . ! ? 제외한 모든 문자 제거
df_train['condition']=df_train['condition'].apply(lambda x: re.sub('[^a-zA-Z.!?]',' ',x))
df_test['condition']=df_test['condition'].apply(lambda x: re.sub('[^a-zA-Z.!?]',' ',x))
df_train['review']=df_train['review'].apply(lambda x: re.sub('[^a-zA-Z.!?]',' ',x))
df_test['review']=df_test['review'].apply(lambda x: re.sub('[^a-zA-Z.!?]',' ',x))

In [ ]:
### Review를 Lemmatization 
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer=WordNetLemmatizer()

lem_train = df_train.copy()
lem_test = df_test.copy()

lem_train['review'] = df_train['review'].map(lambda x: ' '.join(lemmatizer.lemmatize(word, pos=get_wordnet_pos(word)) for word in word_tokenize(x)))
lem_test['review'] = df_test['review'].map(lambda x: ' '.join(lemmatizer.lemmatize(word, pos=get_wordnet_pos(word)) for word in word_tokenize(x)))

In [ ]:
### 불용어 제거
lem_train['review']=lem_train['review'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))
lem_test['review']=lem_test['review'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

In [ ]:
lem_train

,uniqueID,drugName,condition,review,rating,date,usefulCount,review2
0,206461,Valsartan,left ventricular dysfunction,it have no side effect i take it in combinatio...,9,20-May-12,27,side effect take combination bystolic mg fish oil
1,95260,Guanfacine,adhd,my son be halfway through his fourth week of i...,8,27-Apr-10,192,son halfway fourth week intuniv . become conce...
2,92703,Lybrel,birth control,i use to take another oral contraceptive which...,5,14-Dec-09,17,use take another oral contraceptive pill cycle...
3,138000,Ortho Evra,birth control,this be my first time use any form of birth co...,8,3-Nov-15,10,first time use form birth control . glad go pa...
4,35696,Buprenorphine / naloxone,opiate dependence,suboxone have completely turn my life around ....,9,27-Nov-16,37,suboxone completely turn life around . feel he...
...,...,...,...,...,...,...,...,...
161292,191035,Campral,alcohol dependence,i write my first report in mid october of . i ...,10,31-May-15,125,write first report mid october . alcohol since...
161293,127085,Metoclopramide,nausea vomiting,i be give this in iv before surgey . i immedia...,1,1-Nov-11,34,give iv surgey . immediately become anxious co...
161294,187382,Orencia,rheumatoid arthritis,limited improvement after month developed bad ...,2,15-Mar-14,35,limited improvement month developed bad rash m...
161295,47128,Thyroid desiccated,underactive thyroid,i ve be on thyroid medication year i spent my ...,10,19-Sep-15,79,thyroid medication year spent first synthroid ...


In [ ]:
### 2단어 이상 condition 1단어로 합치기
concon=pd.read_csv('https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/dataset/spaced_conidtion.csv', index_col=0)
ori_con=concon['original'].unique()
mod_con=concon['modified'].unique()

for i in range(len(ori_con)):
    ori_con[i]=re.sub('[^a-zA-Z]',' ',ori_con[i])

for i in range(len(mod_con)):
    mod_con[i]=re.sub('[^a-zA-Z]','',mod_con[i])

con_dict={}
for i in range(len(mod_con)):
    con_dict[ori_con[i]]=mod_con[i]

print(con_dict)

{'left ventricular dysfunction': 'leftventriculardysfunction', 'adhd': 'adhd', 'birth control': 'birthcontrol', 'opiate dependence': 'opiatedependence', 'benign prostatic hyperplasia': 'benignprostatichyperplasia', 'emergency contraception': 'emergencycontraception', 'bipolar disorde': 'bipolardisorde', 'epilepsy': 'epilepsy', 'migraine prevention': 'migraineprevention', 'depression': 'depression', 'crohn s disease': 'crohnsdisease', 'cough': 'cough', 'obesity': 'obesity', 'urinary tract infection': 'urinarytractinfection', 'ibromyalgia': 'ibromyalgia', 'chronic myelogenous leukemia': 'chronicmyelogenousleukemia', 'hiv infection': 'hivinfection', 'insomnia': 'insomnia', 'rheumatoid arthritis': 'rheumatoidarthritis', 'vaginal yeast infection': 'vaginalyeastinfection', 'chlamydia infection': 'chlamydiainfection', 'hirsutism': 'hirsutism', 'panic disorde': 'panicdisorde', 'migraine': 'migraine', 'pain': 'pain', 'irritable bowel syndrome': 'irritablebowelsyndrome', 'osteoarthritis': 'osteo

In [ ]:
### review와 condition에 적용

con_re = re.compile('(%s)' % '|'.join(con_dict.keys()))

def expand_con(s, con_dict=con_dict):
    def replace(match):
        return con_dict[match.group(0)]
    return con_re.sub(replace, s)

lem_train['review'] = lem_train['review'].apply(lambda x: expand_con(x))
lem_test['review'] = lem_test['review'].apply(lambda x: expand_con(x))

lem_train['condition']=lem_train['condition'].apply(lambda x: expand_con(x))
lem_test['condition']=lem_test['condition'].apply(lambda x: expand_con(x))

In [ ]:
### 엑셀파일로 저장

!pip install xlsxwriter
lem_train.to_excel('lem_train.xlsx', engine='xlsxwriter')
lem_test.to_excel('lem_test2.xlsx', engine='xlsxwriter')